In [1]:
!pip install ipython-autotime
%matplotlib inline
%load_ext autotime
%matplotlib inline

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-py3-none-any.whl size=1830 sha256=ec99e6337fe7117a4309e670fd1a48395f1a6e781a044ca2e7e59fa637f2c850
  Stored in directory: /root/.cache/pip/wheels/28/3a/38/7ecc013df8b58518cf859fb86c67c53df318426e96bb9fe1c0
Successfully built ipython-autotime


In [2]:
# !pip install googletrans
# !pip install ipython-autotime
# !pip install bert-for-tf2|
# !pip install sentencepiece

time: 386 µs


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# from googletrans import Translator
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# import bert
# FullTokenizer = bert.bert_tokenization.FullTokenizer
import os,re
import matplotlib
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

time: 6.45 s


In [4]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)


time: 3.22 s


In [5]:
data_path = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/'

# loading all the train datasets

# train_data1 = pd.read_csv(data_path + 'jigsaw-toxic-comment-train.csv')
train_data2 = pd.read_csv(data_path + 'jigsaw-toxic-comment-train-processed-seqlen128.csv')
# train_data3 = pd.read_csv(data_path + 'jigsaw-unintended-bias-train.csv')
# train_data4 = pd.read_csv(data_path + 'jigsaw-unintended-bias-train-processed-seqlen128.csv')

# loading all the validation and test datasets

# validation_data1 = pd.read_csv(data_path +  'validation.csv')
validation_data2 = pd.read_csv(data_path + 'validation-processed-seqlen128.csv')

# test_data1 = pd.read_csv(data_path +  'test.csv')
test_data2 = pd.read_csv(data_path + 'test-processed-seqlen128.csv')


time: 5.31 s


In [6]:
# test_data2.head()

time: 383 µs


In [7]:
# train_data2=list(train_data2)
# validation_data2=list(validation_data2)

time: 333 µs


In [8]:
vocab_size = 300209
embedding_dim = 16
max_length = 128
trunc_type='post'

time: 707 µs


In [9]:
# def preprocess_sentence(w):
#     # creating a space between a word and the punctuation following it
#     w = re.sub(r"([?.!,¿])", r" \1 ", w)
#     w = re.sub(r'[" "]+', " ", w)

#     # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
#     w = re.sub(r"[^a-zA-Z?.!,]+", " ", w)
#     #w = re.sub(r"😊"," ",w)
#     w = w.strip()

#     # adding a start and an end token to the sentence
#     # so that the model know when to start and stop predicting.
#     return w

# preprocess_sentence_vf = np.vectorize(preprocess_sentence)


time: 491 µs


In [10]:
train_labels = np.asarray([train_data2.iloc[:223520,2]])
valid_labels = np.asarray([validation_data2.iloc[:,3]])
train_labels.shape

(1, 223520)

time: 108 ms


In [11]:
c = 5672
print(list(eval(train_data2['input_word_ids'][c])))
print(list(eval(validation_data2['input_word_ids'][c])))

[101, 91327, 10107, 106, 10882, 13028, 10944, 41549, 117, 146, 65000, 112, 188, 20378, 169, 19826, 10108, 11424, 10169, 41163, 10107, 11360, 119, 11065, 10529, 19034, 10590, 169, 42126, 15217, 132, 146, 12888, 10189, 13028, 13446, 11222, 10435, 10108, 10105, 20255, 10169, 14091, 10858, 10526, 10111, 11019, 10105, 41163, 10107, 13448, 117, 10380, 146, 10944, 21964, 10105, 41163, 10107, 24087, 10111, 23195, 10135, 19509, 119, 49983, 13172, 72894, 74755, 89771, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[101, 10734, 100025, 10113, 12074, 119, 25732, 119, 16628, 119, 21717, 117, 60160, 10138, 12843, 41121, 10138, 12843, 25921, 10305, 49869, 103148, 10161, 70653, 10123, 113, 49869, 103148, 10161, 11794, 320, 31285, 38711, 10138, 114, 19165, 23763, 13369, 117, 31826, 51484, 32212, 13587, 176, 101535, 33694, 10627, 50302, 27295, 39757, 175, 10729, 189, 22953, 

In [12]:
valid2_comments_arr=validation_data2['input_word_ids'].tolist()
train2_comments_arr=train_data2['input_word_ids'].tolist()

def obj_tolist(valid2_comments_arr):
    valid2_comments_arr_mod=[]
    for ix,x in enumerate(valid2_comments_arr):
        valid2_comments_arr_mod.append(list(eval(str(x))))
    return valid2_comments_arr_mod

valid2_comments_arr_list=obj_tolist(valid2_comments_arr)
train2_comments_arr_list=obj_tolist(train2_comments_arr)

train=np.array(train2_comments_arr_list)
valid=np.array(valid2_comments_arr_list)
train=train[:223520,:]



time: 51.5 s


In [13]:
print(train.shape)
valid.shape
      

(223520, 128)


(8000, 128)

time: 2.63 ms


In [14]:
# train1_comments_arr=preprocess_sentence_vf(train1_comments_arr)
# valid1_comments_arr=preprocess_sentence_vf(valid1_comments_arr)

# print(train1_comments_arr.shape)
# valid1_comments_arr.shape

time: 383 µs


In [15]:
# try:
#     del(translator)
# except:
#     pass
# translator = Translator()
# def translate(statement):
#     try:
#         print(translator.translate(statement,dest='en').text)
#         return translator.translate(statement,dest='en').text
#     except:
#         print("<NO TRANSLATION>"+str(statement)+"<NO TRANSLATION>")
#         return "<NO TRANSLATION>"+str(statement)+"<NO TRANSLATION>"
#         pass
    
# translate_vf=np.vectorize(translate)

time: 502 µs


In [16]:
# train1_comments_arr=translate_vf(train1_comments_arr[:100])
# valid1_comments_arr_trans=translate_vf(valid1_comments_arr[:5000])



time: 357 µs


In [17]:
# def tokenizer_f(statements):
#     tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
#     tokenizer.fit_on_texts(statements)
#     word_idx=tokenizer.word_index
#     return word_idx,tokenizer

# def pad_seq(statements,tokenizer):
#     global max_length
#     global trunc_type
#     sequences = tokenizer.texts_to_sequences(statements)
#     padded = pad_sequences(sequences, maxlen=128, padding='post',truncating='post')
#     return padded

time: 426 µs


In [18]:
# word_idx,tokenizer=tokenizer_f(train2_comments_arr)
# print(len(word_idx))
# train_final_pad=pad_seq(train2_comments_arr,tokenizer)
# print(train_final_pad.shape)

# valid_final_pad=pad_seq(valid2_comments_arr,tokenizer)
# print(valid_final_pad.shape)


time: 364 µs


In [19]:
# reverse_word_index = dict([(value, key) for (key, value) in tokenizer.word_index.items()])

# def decode_review(text):
#     return ' '.join([reverse_word_index.get(i, '?') for i in text])

# print(decode_review(train_final_pad[3]))
# print(train1_comments_arr[3])

time: 361 µs


In [20]:
with tpu_strategy.scope():
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu',
                          kernel_regularizer=tf.keras.regularizers.l2(0.01),
                            activity_regularizer=tf.keras.regularizers.l1(0.01)),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 128, 32)           9606688   
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                2112      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 9,608,865
Trainable params: 9,608,865
Non-trainable params: 0
_________________________________________________________________
time: 667 ms


In [21]:
num_epochs = 2
model.fit(train, train_labels.T, epochs=num_epochs, validation_data=(valid, valid_labels.T))

Train on 223520 samples, validate on 8000 samples
Epoch 1/2
223520/223520 [==============================] - 74s 331us/sample - loss: 0.2572 - accuracy: 0.9122 - val_loss: 0.4591 - val_accuracy: 0.8478
Epoch 2/2
223520/223520 [==============================] - 70s 314us/sample - loss: 0.1751 - accuracy: 0.9413 - val_loss: 0.5995 - val_accuracy: 0.8416


time: 2min 25s


In [22]:
# model.save('/kaggle/working/simple-flatten-84.0-word_idsacc.h5')

time: 425 µs


In [23]:
test_data2_arr=test_data2['input_word_ids'].tolist()

test_data2_arr_list=obj_tolist(test_data2_arr)

test=np.array(test_data2_arr_list)



time: 14.5 s


In [24]:
prediction=model.predict(test)

time: 49.8 s


In [25]:
pred_id=test_data2['id']

time: 849 µs


In [26]:
data_tuples = list(zip(pred_id,prediction.reshape((63812,)).tolist()))
data_tuples

[(0, 0.0038988590240478516),
 (1, 9.688735008239746e-05),
 (2, 0.002144157886505127),
 (3, 0.00015291571617126465),
 (4, 0.008967787027359009),
 (5, 0.028641223907470703),
 (6, 0.0007081329822540283),
 (7, 0.007903814315795898),
 (8, 0.033207327127456665),
 (9, 0.7302263975143433),
 (10, 0.0003166496753692627),
 (11, 0.03785988688468933),
 (12, 0.29003986716270447),
 (13, 0.0008693039417266846),
 (14, 0.051025211811065674),
 (15, 0.04965922236442566),
 (16, 0.00022745132446289062),
 (17, 9.179115295410156e-05),
 (18, 0.06070861220359802),
 (19, 0.02018815279006958),
 (20, 0.00027945637702941895),
 (21, 0.0002785027027130127),
 (22, 0.010568678379058838),
 (23, 0.02178668975830078),
 (24, 0.011728048324584961),
 (25, 0.7150291204452515),
 (26, 0.013093948364257812),
 (27, 0.0002957582473754883),
 (28, 0.0003412961959838867),
 (29, 0.01265573501586914),
 (30, 0.02657821774482727),
 (31, 0.00027808547019958496),
 (32, 0.05976727604866028),
 (33, 0.009879469871520996),
 (34, 0.002544730901

time: 92.4 ms


In [27]:
test_df=pd.DataFrame(data_tuples, columns=['id','toxic'])
test_df.to_csv("submission.csv",index=False)

time: 604 ms
